In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer#missing valu easily handle
from sklearn.pipeline import Pipeline

#regression

from sklearn.linear_model import LinearRegression,Ridge
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor,StackingRegressor

#metrics

from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import warnings

warnings.filterwarnings("ignore")


In [2]:
df=pd.read_csv('student.csv')
df.head(10)

,date,gender,age,address,famsize,Pstatus,M_Edu,F_Edu,M_Job,F_Job,relationship,smoker,tuition_fee,time_friends,ssc_result,hsc_result
0,29/04/2018,M,18,Rural,GT3,Together,3,2,At_home,Farmer,No,No,71672,4,4.22,3.72
1,29/04/2018,F,19,Rural,LE3,Apart,0,4,Other,Health,Yes,No,26085,5,3.47,2.62
2,29/04/2018,F,19,Rural,GT3,Together,0,3,Teacher,Services,No,No,40891,3,3.32,2.56
3,29/04/2018,F,19,Rural,LE3,Apart,2,3,At_home,Business,No,No,50600,2,4.57,4.17
4,29/04/2018,M,17,Rural,GT3,Together,1,1,At_home,Farmer,No,No,62458,2,4.50,3.94
5,29/04/2018,M,18,Rural,GT3,Together,2,4,At_home,Farmer,No,Yes,43645,2,4.09,3.65
6,29/04/2018,M,18,Urban,GT3,Together,1,0,Teacher,Teacher,No,No,111578,1,3.46,2.99
7,29/04/2018,F,18,Rural,GT3,Together,2,1,Services,Teacher,Yes,No,76596,3,4.51,3.92
8,29/04/2018,F,17,Rural,GT3,Together,2,3,Other,Teacher,Yes,Yes,39874,3,3.26,2.59
9,29/04/2018,F,17,Rural,LE3,Together,2,2,Health,Health,No,No,48767,3,4.72,4.25


# Y DAta profiling

In [3]:
df.shape

(2018, 16)

In [4]:
from ydata_profiling import ProfileReport
profile=ProfileReport(df,title="Bangladesh student performance ",explorative=True)
profile.to_file("ydata.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 16/16 [00:00<00:00, 57.76it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Distinct means 100 percent means here is not repeated value but we want to 

repeated value because our model learn pattern by using this pattern 

low correlation is not always useless data here is include some  correlarion 

here .

machine learning model a less correlation feature we take this low correlation pattern model also 

class imbalance formula: if any feature have more value less than another value 

our model wiil be find this more value for prediction but we dont want this 

like this if there are 3 types of value like hsc is 1800 ,ssc is 200,jsc is 100 

our model predict hsc more than ssc and jsc this is class imbalance we handle 

this by using statify formula when we do train test split same distribution 

# Basic EDA

In [5]:
len(df.columns)

16

In [6]:
df.columns

Index(['date', 'gender', 'age', 'address', 'famsize', 'Pstatus', 'M_Edu',
       'F_Edu', 'M_Job', 'F_Job', 'relationship', 'smoker', 'tuition_fee',
       'time_friends', 'ssc_result', 'hsc_result'],
      dtype='object')

# Simplify Columns name

In [7]:
df.columns =[
    "date","st_gender",'st_age',"st_address",'st_fam_size','pstatus','m_edu',
    'f_edu','m_job','f_job',"st_relationship",'smoker','tuition_fee','time_with_friends',
    'ssc_result','hsc_result'
]

In [8]:
df.columns

Index(['date', 'st_gender', 'st_age', 'st_address', 'st_fam_size', 'pstatus',
       'm_edu', 'f_edu', 'm_job', 'f_job', 'st_relationship', 'smoker',
       'tuition_fee', 'time_with_friends', 'ssc_result', 'hsc_result'],
      dtype='object')

# Drop date

In [9]:
if "date" in df.columns:
    df.drop(columns=['date'],inplace=True)

In [10]:
df.head(10)

,st_gender,st_age,st_address,st_fam_size,pstatus,m_edu,f_edu,m_job,f_job,st_relationship,smoker,tuition_fee,time_with_friends,ssc_result,hsc_result
0,M,18,Rural,GT3,Together,3,2,At_home,Farmer,No,No,71672,4,4.22,3.72
1,F,19,Rural,LE3,Apart,0,4,Other,Health,Yes,No,26085,5,3.47,2.62
2,F,19,Rural,GT3,Together,0,3,Teacher,Services,No,No,40891,3,3.32,2.56
3,F,19,Rural,LE3,Apart,2,3,At_home,Business,No,No,50600,2,4.57,4.17
4,M,17,Rural,GT3,Together,1,1,At_home,Farmer,No,No,62458,2,4.50,3.94
5,M,18,Rural,GT3,Together,2,4,At_home,Farmer,No,Yes,43645,2,4.09,3.65
6,M,18,Urban,GT3,Together,1,0,Teacher,Teacher,No,No,111578,1,3.46,2.99
7,F,18,Rural,GT3,Together,2,1,Services,Teacher,Yes,No,76596,3,4.51,3.92
8,F,17,Rural,GT3,Together,2,3,Other,Teacher,Yes,Yes,39874,3,3.26,2.59
9,F,17,Rural,LE3,Together,2,2,Health,Health,No,No,48767,3,4.72,4.25


Correlation for numerical values 

In [11]:
corr_target=df.select_dtypes(include=np.number).corr()['hsc_result'].sort_values(ascending=False)

corr_target

hsc_result           1.000000
ssc_result           0.950178
m_edu                0.063776
f_edu                0.054811
tuition_fee          0.038068
st_age              -0.009857
time_with_friends   -0.156356
Name: hsc_result, dtype: float64

In [12]:
x=df.drop("hsc_result",axis=1)
y=df['hsc_result']

y

0       3.72
1       2.62
2       2.56
3       4.17
4       3.94
        ... 
2013    3.18
2014    3.13
2015    3.15
2016    2.83
2017    3.15
Name: hsc_result, Length: 2018, dtype: float64

In [13]:
x

,st_gender,st_age,st_address,st_fam_size,pstatus,m_edu,f_edu,m_job,f_job,st_relationship,smoker,tuition_fee,time_with_friends,ssc_result
0,M,18,Rural,GT3,Together,3,2,At_home,Farmer,No,No,71672,4,4.22
1,F,19,Rural,LE3,Apart,0,4,Other,Health,Yes,No,26085,5,3.47
2,F,19,Rural,GT3,Together,0,3,Teacher,Services,No,No,40891,3,3.32
3,F,19,Rural,LE3,Apart,2,3,At_home,Business,No,No,50600,2,4.57
4,M,17,Rural,GT3,Together,1,1,At_home,Farmer,No,No,62458,2,4.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,M,18,Urban,GT3,Together,2,2,Teacher,Health,No,No,108426,2,3.57
2014,M,19,Urban,GT3,Together,0,3,Other,Teacher,Yes,No,100665,5,4.05
2015,M,18,Urban,GT3,Together,3,2,Services,Farmer,Yes,No,92291,1,3.93
2016,M,19,Rural,GT3,Together,0,2,At_home,Teacher,No,No,43926,3,3.34


In [14]:
#for numerical 

num_transformer=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy="median")),
        ("scaler",StandardScaler())
    ]
)

#for categorical 

cat_transformer=([
        ('imputer',SimpleImputer(strategy="most_frequent")),
        ("encoder",OneHotEncoder(handle_unknown="ignor"))
        #dhaka ,sylhet error gula ke ignor korbe
    ]
)

In [15]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

# Ensemble learner

In [16]:
reg_lr=LinearRegression()
reg_rf=RandomForestRegressor(n_estimators=100,random_state=42)
reg_gb=GradientBoostingRegressor(n_estimators=100,random_state=42)

# Voting regressor

In [17]:
voting_reg=VotingRegressor(
    estimators=[
        ("lr",reg_lr),
        ('rf',reg_rf),
        ("gb",reg_gb)
    ]
)

# Stacking

In [18]:
stacking_reg=StackingRegressor(
    estimators=[('rf',reg_rf),

],
final_estimator=Ridge()
)

# dictionary of all model